In [1]:
import os
import sys
print(sys.path)
import BioSimSpace as BSS

['/opt/miniforge/envs/openbiosim/lib/python39.zip', '/opt/miniforge/envs/openbiosim/lib/python3.9', '/opt/miniforge/envs/openbiosim/lib/python3.9/lib-dynload', '', '/home/purohit.52@pharmacy.ohio-state.edu/.local/lib/python3.9/site-packages', '/opt/miniforge/envs/openbiosim/lib/python3.9/site-packages']


INFO:rdkit:Enabling RDKit 2023.09.6 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
main_folder = os.getcwd()
input_dir = f"{main_folder}/inputs"
path_to_ligands = f"{main_folder}/inputs/ligands"
path_to_protein = f"{main_folder}/inputs"
# make a directory for all our output files
if not os.path.exists(f"{main_folder}/output_setup"):
    os.mkdir(f"{main_folder}/output_setup")

In [ ]:
MG = BSS.IO.readMolecules(["inputs/ligands/magnesium.prm7","inputs/ligands/magnesium.rst7"])[0]   
BSS.IO.saveMolecules("MG", MG, ["gro87", "grotop"])

In [3]:
# atp = BSS.IO.readMolecules(["inputs/ligands/atp_maegher.prm7","inputs/ligands/atp_maegher.rst7"])[0]   
# adp = BSS.IO.readMolecules(["inputs/ligands/adp_maegher.prm7","inputs/ligands/adp_maegher.rst7"])[0]   

In [7]:
# atp = BSS.IO.readMolecules(["inputs/ligands/gtp_maegher.prm7","inputs/ligands/gtp_maegher.rst7"])[0]   
# adp = BSS.IO.readMolecules(["inputs/ligands/gdp_maegher.prm7","inputs/ligands/gdp_maegher.rst7"])[0]   

In [8]:
atp = BSS.IO.readMolecules(["inputs/ligands/ATP-charmm36/topol.top","inputs/ligands/ATP-charmm36/step3_input.gro"])[0]  
adp = BSS.IO.readMolecules(["inputs/ligands/ADP-charmm36/topol.top","inputs/ligands/ADP-charmm36/step3_input.gro"])[0]  

In [9]:
mapping = BSS.Align.matchAtoms(atp, adp)
# Mapping is a dictionary mapping atom indices in ethane to those in methanol.
print(mapping)
inv_mapping = {v: k for k, v in mapping.items()}
adp_aligned = BSS.Align.rmsdAlign(adp, atp, inv_mapping)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 37, 37: 36, 38: 38}


In [10]:
merged = BSS.Align.merge(atp, adp_aligned, mapping)
solvated = BSS.Solvent.tip3p(molecule=merged, box=3*[50*BSS.Units.Length.angstrom], ion_conc=0.15, is_neutral=True)
protocol = BSS.Protocol.FreeEnergyProduction(timestep=2*BSS.Units.Time.femtosecond, runtime=0.001*BSS.Units.Time.nanosecond, num_lam=3)
fep_free = BSS.FreeEnergy.Relative(solvated, protocol, engine="GROMACS", work_dir="ligandleg_atp_adp_noMG", ignore_warnings=True, show_errors=True)
fep_free.run()
print("done")

done


In [60]:
#complx = merged + protein

In [10]:
#2*[140*BSS.Units.Length.angstrom]

[140.0000 A, 140.0000 A]

In [61]:
complex_sol = BSS.Solvent.tip3p(molecule=complx, box=[88*BSS.Units.Length.angstrom, 138*BSS.Units.Length.angstrom, 138*BSS.Units.Length.angstrom], ion_conc=0.15, is_neutral=True)
protocol = BSS.Protocol.FreeEnergy(runtime=0.001*BSS.Units.Time.nanosecond, num_lam=5)
fep_bound = BSS.FreeEnergy.Relative(complex_sol, protocol, engine="GROMACS", ignore_warnings=True, work_dir="atp_adp_3ice_E/bound")

fep_bound.run()  
fep_bound.wait()
print("done")

done


In [17]:
complex_sol = BSS.Solvent.tip3p(molecule=complx, box=3*[140*BSS.Units.Length.angstrom], ion_conc=0.15, is_neutral=True)
protocol = BSS.Protocol.FreeEnergy(runtime=0.001*BSS.Units.Time.nanosecond, num_lam=5)
fep_bound = BSS.FreeEnergy.Relative(complex_sol, protocol, engine="GROMACS", ignore_warnings=True, work_dir="atp_adp_3ice_F/bound")

fep_bound.run()  
fep_bound.wait()
print("done")

done


In [15]:
# Solvate the protein ligand complex in a 60 angstrom box of TIP3P water.
#complex_sol = BSS.Solvent.tip3p(molecule=complx, box=3*[130*BSS.Units.Length.angstrom], ion_conc=0.15, is_neutral=True)
# Solvate the merged ligand in a 60 angstrom box of TIP3P water.
#merged_sol = BSS.Solvent.tip3p(molecule=merged, box=3*[60*BSS.Units.Length.angstrom])

# Create the free energy protocol.
#protocol = BSS.Protocol.FreeEnergy(runtime=0.01*BSS.Units.Time.nanosecond, num_lam=9)

# Initialise relative free energy objects for each leg.
# (Once again, this defaults to the SOMD engine.)
#fep_bound = BSS.FreeEnergy.Relative(complex_sol, protocol, engine="GROMACS", ignore_warnings=True, work_dir="gtp_gdp_gmx/bound")

#fep_bound = BSS.FreeEnergy.Relative(complex_sol, protocol, engine="GROMACS", ignore_warnings=True, work_dir="atp_adp_A/bound")
#fep_bound = BSS.FreeEnergy.Relative(complex_sol, protocol, engine="GROMACS", ignore_warnings=True, work_dir="atp_adp_B/bound")

#fep_free  = BSS.FreeEnergy.Relative(merged_sol, protocol, work_dir="gtp_gdp/free")

In [16]:
fep_bound.run()  
fep_bound.wait()

In [17]:
print("done")

done


In [63]:
#solvated = BSS.Solvent.tip3p(molecule=merged, box=3*[40*BSS.Units.Length.angstrom], ion_conc=0.15, is_neutral=True)
#BSS.Notebook.View(solvated).system()
#BSS.IO.savePerturbableSystem("pert", solvated)
#! ls pert*
#solvated = BSS.IO.readPerturbableSystem("pert0.prm7", "pert0.rst7", "pert1.prm7", "pert1.rst7")

Next we can setup a free energy calculation Protocol to compute the free energy change for morphing ethane into methanol in vacuum using BSS.Protocol.FreeEnergy. Here we use a small number of lambda windows and a short run time to complete the calculation rapidly. We use default settings for most parameters, but tweak the frequency at which simulation data is saved to reduce the footprint of the calculation.

We can now create a process to run a free energy calculation by combining the protocol with a simulation engine using BSS.FreeEnergy.Relative. Note that the interface is agnostic to the choice of the simulation engine. The same protocol and merged molecule can be used to execute the free energy calculation using different engines. Here we use the engine SOMD but GROMACS could be used by updating just one keyword argument passed to BSS.FreeEnergy.Relative. Behind the scenes BSS will translate the specified BSS.Protocol into engine specific inputs.

In [ ]:
# Let's run this calculation. This may take a few minutes 
protocol = BSS.Protocol.FreeEnergy(timestep=2*BSS.Units.Time.femtosecond, runtime=4*BSS.Units.Time.nanosecond, num_lam=16)
fep_vac  = BSS.FreeEnergy.Relative(merged.toSystem(), protocol, engine="GROMACS", work_dir="atp_adp_gmx/vacuum", ignore_warnings=False, show_errors=True)
fep_vac.run()  
fep_vac.wait()  

#Now that the simulation has completed we can analyse the generated trajectories to obtain
#the free energy change by using Pymbar. Note again that the analysis call is agnostic to the chosen simulation engine.

In [ ]:
pmf_vac, overlap_vac = BSS.FreeEnergy.Relative.analyse(f'atp_adp_gmx/vacuum')
print ("The free energy change is DG_vac = %s +/- %s"% (pmf_vac[-1][1], pmf_vac[-1][2]))

In [62]:
solvated = BSS.Solvent.tip3p(molecule=merged, box=3*[40*BSS.Units.Length.angstrom], ion_conc=0.15, is_neutral=True)

protocol = BSS.Protocol.FreeEnergyProduction(timestep=2*BSS.Units.Time.femtosecond, runtime=0.01*BSS.Units.Time.nanosecond, num_lam=5)
fep_free = BSS.FreeEnergy.Relative(solvated, protocol, engine="GROMACS", work_dir="atp_adp_gmx/free", ignore_warnings=True, show_errors=True)
fep_free.run()
print("done")

ThemeManager()

pert0.prm7  pert0.rst7	pert1.prm7  pert1.rst7


In [63]:
protocol = BSS.Protocol.FreeEnergyProduction(timestep=2*BSS.Units.Time.femtosecond, runtime=0.01*BSS.Units.Time.nanosecond, num_lam=5)
fep_free = BSS.FreeEnergy.Relative(solvated, protocol, engine="GROMACS", work_dir="atp_adp_gmx/free", ignore_warnings=True, show_errors=True)
fep_free.run()
fep_free.wait()

In [11]:
pmf_free, overlap_free  = fep_free.analyse()
free_nrg_solvation = BSS.FreeEnergy.Relative.difference(pmf_free, pmf_vac)
print(free_nrg_solvation)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 pmf_free, overlap_free  = fep_free.analyse()                                                 │
│ ❱ 2 free_nrg_solvation = BSS.FreeEnergy.Relative.difference(pmf_free, pmf_vac)                   │
│   3 print(free_nrg_solvation)                                                                    │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'pmf_vac' is not defined

In [91]:
pmf_vac, discard = BSS.FreeEnergy.Relative.analyse(f'atp_adp_gmx/free')
print ("The free energy change is DG_sol = %s +/- %s"% (pmf_vac[-1][1], pmf_vac[-1][2]))

The free energy change is DG_sol = 240.0542 kcal/mol +/- 17.1401 kcal/mol
